In [1]:
from neo4j import GraphDatabase, basic_auth
import os

driver = GraphDatabase.driver("bolt://localhost:7687",auth=basic_auth("neo4j", "Kaffe1234"))

In [2]:
def get_count_of_type(label, session):
    q = "MATCH (n:%s) RETURN count(n)" % label
    res = session.run(q)
    print("Added", res.value()[0], "nodes of type", label)
    
def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:%s]-() RETURN count(*)" %label
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/regions")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [22]:
# Deleting all region data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE n:BrainRegion 
                OR n:RegionZone 
                OR n:RegionRecord 
            DETACH DELETE n
        ''')
        print("wipeddatabase")

wipeddatabase


In [23]:
# Adding region to graph with BrainRegion label
csv_file_path = "file:///%s" % get_csv_path("regions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:BrainRegion {id: row.ID, name: row.Region_name, preferred: 1})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("BrainRegion", session)

Added 30 nodes of type BrainRegion


In [24]:
# Adding regions_other to graph with BrainRegion label
csv_file_path = "file:///%s" % get_csv_path("regions_other.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:BrainRegion {id: row.ID, name: row.Region_name, abbreviation: row.Abbreviation, comments: row.Comments, preferred: 0})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("BrainRegion", session)

Added 302 nodes of type BrainRegion


In [25]:
# Adding region_records to graph with RegionRecord label
csv_file_path = "file:///%s" % get_csv_path("region_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:RegionRecord {id: row.ID, name: row.Region_record_name, coverage: row.Coverage, specificity: row.Specificity, no_original_regions: row.No_original_regions, original_region_retained: row.Original_region_retained, parcellation_scheme: row.Parcellation_scheme, atlas_coordinates: row.Atlas_coordinates, illustration: row.Illustration,semantic_description: row.Semantic_description, annotated_images: row.Annotated_images,regional_characteristics: row.Regional_characteristics,atlas_reg: row.Atlas_reg,serial_sections: row.Serial_sections,collectors_comment: row.Collectors_comment,original_framework: row.Original_framework,documentation_score: row.Documentation_score})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("RegionRecord", session)

Added 313 nodes of type RegionRecord


In [26]:
# Relationship PRIMARY_REGION between BrainRegion and RegionRecord
csv_file_path = "file:///%s" % get_csv_path("region_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionRecord {id: row.ID})
        MATCH (c:BrainRegion {id: row.Region, preferred: 1 })
        MERGE (a)-[:PRIMARY_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PRIMARY_REGION", session)

Added 624 relationships of type PRIMARY_REGION


In [27]:
# Relationship SECONDARY_REGION between BrainRegion and RegionRecord
csv_file_path = "file:///%s" % get_csv_path("region_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionRecord {id: row.ID})
        MATCH (c:BrainRegion {id: row.Secondary_region, preferred: 0 })
        MERGE (a)-[:SECONDARY_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SECONDARY_REGION", session)

Added 36 relationships of type SECONDARY_REGION
